## Load Fin-Fan failure detail

In [1]:
#Dependencies
import pandas as pd
from sqlalchemy import create_engine



Extract CSV into DataFrames

In [2]:
failure_csv = "resources/HVAC_failures_2021.csv"

Transform CSV DataFrame

In [3]:
failure_df = pd.read_csv(failure_csv)
failure_df.head()


,Event name,Start time,End time,Duration,Event template,Primary element,Primary element path,Running status reset post event start?,Unnamed: 8,Difference,Comments,Failure
0,HVAC Fan S311HHU205B Bearing Failure 2020-01-1...,16/01/2020 14:11,16/01/2020 14:20,0:08:54,HVAC Fan Failed Event Frame,S311HHU205B,\\ICHTHYS_PIAF\Predictive Analytics\HVAC Fan\F...,9/02/2020 15:41,0.0,577.3433419,False Alarm (speed drop for few mins),False
1,HVAC Fan S311HHU209B Bearing Failure 2020-03-0...,8/03/2020 20:11,8/03/2020 20:51,0:40:10,HVAC Fan Failed Event Frame,S311HHU209B,\\ICHTHYS_PIAF\Predictive Analytics\HVAC Fan\F...,26/03/2020 22:39,0.0,433.8001192,False Alarm (speed drop for few mins),False
2,HVAC Fan S311HHU209B Bearing Failure 2020-04-2...,24/04/2020 10:02,24/04/2020 10:22,0:19:45,HVAC Fan Failed Event Frame,S311HHU209B,\\ICHTHYS_PIAF\Predictive Analytics\HVAC Fan\F...,11/05/2020 9:17,0.0,406.9254994,False Alarm (speed drop for few mins),False
3,HVAC Fan S311HHU209B Bearing Failure 2020-11-0...,5/11/2020 6:35,5/11/2020 6:47,0:11:53,HVAC Fan Failed Event Frame,S311HHU209B,\\ICHTHYS_PIAF\Predictive Analytics\HVAC Fan\F...,31/01/2021 7:18,0.0,2088.51602,False Alarm (speed drop for few mins),False
4,HVAC Fan S311HK017A Bearing Failure 2021-02-06...,6/02/2021 20:31,7/02/2021 7:23,10:52:13,HVAC Fan Failed Event Frame,S311HK017A,\\ICHTHYS_PIAF\Predictive Analytics\HVAC Fan\F...,7/02/2021 7:23,0.0,0,TRUE Failure,True


In [4]:
# Select only the columns required
new_failure_df = failure_df[['Primary element','Start time', 'End time','Duration','Difference','Comments','Failure']].copy()
new_failure_df.head()



,Primary element,Start time,End time,Duration,Difference,Comments,Failure
0,S311HHU205B,16/01/2020 14:11,16/01/2020 14:20,0:08:54,577.3433419,False Alarm (speed drop for few mins),False
1,S311HHU209B,8/03/2020 20:11,8/03/2020 20:51,0:40:10,433.8001192,False Alarm (speed drop for few mins),False
2,S311HHU209B,24/04/2020 10:02,24/04/2020 10:22,0:19:45,406.9254994,False Alarm (speed drop for few mins),False
3,S311HHU209B,5/11/2020 6:35,5/11/2020 6:47,0:11:53,2088.51602,False Alarm (speed drop for few mins),False
4,S311HK017A,6/02/2021 20:31,7/02/2021 7:23,10:52:13,0,TRUE Failure,True


In [5]:
# Rename columns
new_failure_df = new_failure_df.rename(columns={'Primary element':'primary_element', 'Start time': 'start_time', 'End time':'end_time', 'Duration':'duration', 'Difference':'difference', 'Comments':'comments', 'Failure':'failure'})
new_failure_df = new_failure_df.dropna()



In [6]:
# Convert to datetime format for database load
new_failure_df['start_time'] = pd.to_datetime(new_failure_df['start_time'], format='%d/%m/%Y %H:%M')
new_failure_df['end_time'] = pd.to_datetime(new_failure_df['end_time'], format='%d/%m/%Y %H:%M')
new_failure_df.head()

,primary_element,start_time,end_time,duration,difference,comments,failure
0,S311HHU205B,2020-01-16 14:11:00,2020-01-16 14:20:00,0:08:54,577.3433419,False Alarm (speed drop for few mins),False
1,S311HHU209B,2020-03-08 20:11:00,2020-03-08 20:51:00,0:40:10,433.8001192,False Alarm (speed drop for few mins),False
2,S311HHU209B,2020-04-24 10:02:00,2020-04-24 10:22:00,0:19:45,406.9254994,False Alarm (speed drop for few mins),False
3,S311HHU209B,2020-11-05 06:35:00,2020-11-05 06:47:00,0:11:53,2088.51602,False Alarm (speed drop for few mins),False
4,S311HK017A,2021-02-06 20:31:00,2021-02-07 07:23:00,10:52:13,0,TRUE Failure,True


In [8]:
# Drop non numeric 
new_failure_df = new_failure_df[pd.to_numeric(new_failure_df['difference'], errors='coerce').notnull()]
new_failure_df

,primary_element,start_time,end_time,duration,difference,comments,failure
0,S311HHU205B,2020-01-16 14:11:00,2020-01-16 14:20:00,0:08:54,577.3433419,False Alarm (speed drop for few mins),False
1,S311HHU209B,2020-03-08 20:11:00,2020-03-08 20:51:00,0:40:10,433.8001192,False Alarm (speed drop for few mins),False
2,S311HHU209B,2020-04-24 10:02:00,2020-04-24 10:22:00,0:19:45,406.9254994,False Alarm (speed drop for few mins),False
3,S311HHU209B,2020-11-05 06:35:00,2020-11-05 06:47:00,0:11:53,2088.51602,False Alarm (speed drop for few mins),False
4,S311HK017A,2021-02-06 20:31:00,2021-02-07 07:23:00,10:52:13,0,TRUE Failure,True
5,S311HK020A,2020-01-16 14:20:00,2020-01-16 14:27:00,0:07:16,0.240321667,False Alarm (speed drop for few mins),False
6,S311HK049B,2021-02-11 17:08:00,2021-02-11 17:14:00,0:06:00,0,Possible. Refer Trend,True
7,S311HK049B,2021-02-11 18:04:00,2021-02-11 18:10:00,0:06:00,0,Possible. Refer Trend,True
8,S311HK049B,2021-02-11 21:01:00,2021-02-11 21:07:00,0:06:00,0,Possible. Refer Trend,True
9,S311HK049B,2021-02-12 02:46:00,2021-02-12 02:52:00,0:06:00,0,Possible. Refer Trend,True


Load

In [9]:
# PotgreSQL connection requirements
from config import userid
from config import password

In [10]:
# Connect to AWS database
rds_connection_string = f"{userid}:{password}@awspostgres.ctkgxnaawxx6.ap-southeast-2.rds.amazonaws.com:5432/AWSPostgres"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [11]:
engine.table_names()

['hvac_failures']

In [12]:
# Load pandas dataframe to remote Postgres database table 'hvac_failures'
new_failure_df.to_sql(name='hvac_failures', con=engine, if_exists='append', index=False)

In [13]:
# Confirm records in database
from sqlalchemy.orm import Session
from sqlalchemy.ext.automap import automap_base

Base = automap_base()
Base.prepare(engine, reflect=True)
# Base.classes.keys()

failures = Base.classes.hvac_failures

session = Session(engine)

first_row = session.query(failures).first()
first_row.__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x211a52305f8>,
 'id': 39,
 'failure': False,
 'comments': 'False Alarm (speed drop for few mins)',
 'duration': '0:08:54',
 'start_time': datetime.datetime(2020, 1, 16, 14, 11),
 'difference': Decimal('577.3433419'),
 'end_time': datetime.datetime(2020, 1, 16, 14, 20),
 'primary_element': 'S311HHU205B'}